---

## CARGA DE LIBRERÍA Y DATOS

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chi2
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/train.csv')
test = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/sample_submission.csv")

---

## DEFINICIÓN PREPROCESADO

---

In [ ]:
cols_to_int = ['age', 'sex', 'cp', 'restecg']

rename_dict = {
    "age": "edad",
    "sex": "sexo",
    "cp": "tipo_dolor_pecho",
    "trestbps": "tension_en_descanso",
    "chol": "colesterol",
    "fbs": "azucar",
    "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto",
    "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio",
    "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes",
    "thal": "estado_corazon_thal" }

cols_a_clippear = [
    'tension_en_descanso', 'colesterol',
    'latidos_por_minuto', 'cambio_linea_corazon_ejercicio']

categorical_cols_to_round = [
    'num_venas_grandes', 'estado_corazon_thal', 'sexo',
    'tipo_dolor_pecho', 'dolor_pecho_con_ejercicio',
    'azucar', 'forma_linea_corazon_ejercicio', 'electro_en_descanso']

---

FUNCIONES DE PREPROCESADO

---

In [ ]:
def limpieza_inicial(df):
    """
    Realiza conversiones de tipos, renombres y limpieza básica de errores (-9).
    Se puede aplicar a todo el dataset antes del split.
    """
    df = df.copy()

    # Conversión a Int
    for col in cols_to_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    # Conversión de objetos a numérico
    object_cols = df.select_dtypes(include=['object']).columns
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Renombrar
    df = df.rename(columns=rename_dict)

    # Reemplazar errores conocidos (-9) por NaN
    df.replace([-9, -9.0], np.nan, inplace=True)

    return df

In [ ]:
def limpiar_ceros_fisiologicos(X):
    X = X.copy()
    cols_imposibles_con_cero = ['tension_en_descanso', 'colesterol']
    for col in cols_imposibles_con_cero:
        if col in X.columns:
            X[col] = X[col].replace({0: np.nan, 0.0: np.nan})
    return X

In [ ]:
def clipear_outliers(X):
    """
    Nota: Usado en FunctionTransformer, esto calculará los cuantiles
    sobre el lote actual de datos.
    """
    X = X.copy()
    for col in cols_a_clippear:
        if col in X.columns:
            p1 = X[col].quantile(0.01)
            p99 = X[col].quantile(0.99)
            X[col] = X[col].clip(lower=p1, upper=p99)
    return X

In [ ]:
def crear_flags_mnar(df):
    df_new = df.copy()
    cols_mnar = ['num_venas_grandes', 'estado_corazon_thal']
    for col in cols_mnar:
        if col in df_new.columns:
            df_new[f'{col}_is_missing'] = df_new[col].isna().astype(int)
    return df_new

In [ ]:
def redondear_imputaciones(X):
    X = X.copy()
    for col in categorical_cols_to_round:
        if col in X.columns:
            X[col] = X[col].round()
    return X

In [ ]:
def optimizar_k_knn(X_train, k_range=[3, 5, 7, 9, 11, 15]):
    # Preparamos una copia limpia para testear
    X_temp = limpiar_ceros_fisiologicos(X_train)
    X_temp = clipear_outliers(X_temp)
    # Solo usamos filas completas para validar el error de imputación
    X_complete = X_temp.dropna().copy()

    if len(X_complete) < 50:
        print("Pocos datos completos. Se usará k=5 por defecto.")
        return 5

    rmse_scores = {}
    scaler = RobustScaler()
    X_scaled_array = scaler.fit_transform(X_complete)

    np.random.seed(42)
    mask = np.random.rand(*X_scaled_array.shape) < 0.1
    X_missing_sim = X_scaled_array.copy()
    X_missing_sim[mask] = np.nan

    print(f"Buscando k óptimo sobre {len(X_complete)} muestras...")
    for k in k_range:
        imputer = KNNImputer(n_neighbors=k, weights='distance')
        X_imputed = imputer.fit_transform(X_missing_sim)
        error = np.sqrt(mean_squared_error(X_scaled_array[mask], X_imputed[mask]))
        rmse_scores[k] = error

    best_k = min(rmse_scores, key=rmse_scores.get)
    print(f"Mejor k encontrado: {best_k}")
    return best_k

---
## APLICACIÓN DEL FLUJO DE PREPROCESADO DEFINIDO

---

In [ ]:
df_train = df.copy()
df_test = test.copy()

df_train = limpieza_inicial(df_train)
df_test = limpieza_inicial(df_test)

target = "label"

X_train = df_train.drop(columns=target)
y_train = df_train[target]

if target in df_test.columns:
    X_test = df_test.drop(columns=target)
    y_test = df_test[target]
else:
    X_test = df_test.copy()

best_k = optimizar_k_knn(X_train)

pipeline_imputacion = Pipeline([
    ('limpieza_ceros', FunctionTransformer(limpiar_ceros_fisiologicos, validate=False)),
    ('clipear_outliers', FunctionTransformer(clipear_outliers, validate=False)),
    ('mnar_flags', FunctionTransformer(crear_flags_mnar, validate=False)),
    ('scaler', RobustScaler()), # Aprende la mediana y rango intercuartílico de TRAIN
    ('knn_imputer', KNNImputer(n_neighbors=best_k, weights='distance')), # Aprende vecinos de TRAIN
    ('rounding', FunctionTransformer(redondear_imputaciones, validate=False))
]).set_output(transform="pandas")

Buscando k óptimo sobre 246 muestras...
Mejor k encontrado: 15


---
## EJECUCIÓN Y VERIFICACIÓN DE TRANSFORMACIÓN

---

In [ ]:
print("Ajustando pipeline con TRAIN completo...")
X_train_prep = pipeline_imputacion.fit_transform(X_train)

print("Aplicando preprocesamiento a TEST...")
X_test_prep = pipeline_imputacion.transform(X_test)

print("\n--- Proceso finalizado ---")
print(f"Dimensiones Train procesado: {X_train_prep.shape}")
print(f"Dimensiones Test procesado : {X_test_prep.shape}")

print(f"Nulos restantes en Train: {X_train_prep.isna().sum().sum()}")
print(f"Nulos restantes en Test : {X_test_prep.isna().sum().sum()}")

Ajustando pipeline con TRAIN completo...
Aplicando preprocesamiento a TEST...

--- Proceso finalizado ---
Dimensiones Train procesado: (732, 15)
Dimensiones Test procesado : (184, 15)
Nulos restantes en Train: 0
Nulos restantes en Test : 0


---
## ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST CON SVM

---

In [ ]:
#SVM = 0.549

from sklearn.svm import SVC

print("Entrenando SVM con el dataset PREPROCESADO...")
svm_final = SVC(probability=True, random_state=42)

svm_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = svm_final.predict(X_test_prep)


# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_svm_full_5.csv", index=False)
    print("¡Archivo 'submission_svm_full_5.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

Entrenando SVM con el dataset PREPROCESADO...
Generando predicciones sobre el Test...
Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_svm_full_5.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


---
## ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST CON GRADIENT BOOSTING

---

In [ ]:
#Mejoro un 0.1 con preprocesado avanzado (posible descarte)

from sklearn.ensemble import GradientBoostingClassifier

print("Entrenando Gradient Boosting con el dataset PREPROCESADO...")

gb_final = GradientBoostingClassifier(random_state=42)
gb_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = gb_final.predict(X_test_prep)

# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    # Cambiamos el nombre del archivo para distinguirlo del SVM
    submission.to_csv("submission_gb_full_6.csv", index=False)
    print("¡Archivo 'submission_gb_full_6.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

Entrenando Gradient Boosting con el dataset PREPROCESADO...
Generando predicciones sobre el Test...
Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_gb_full_6.csv' guardado con éxito!
   ID  label
0   0      4
1   1      0
2   2      0
3   3      1
4   4      0


---
## ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST CON REGRESIÓN LOGÍSTICA

---

In [ ]:
from sklearn.linear_model import LogisticRegression

print("Entrenando Regresión Logística con el dataset PREPROCESADO...")

lr_final = LogisticRegression(random_state=42, max_iter=1000)
lr_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = lr_final.predict(X_test_prep)

# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    # Nombre de archivo actualizado
    submission.to_csv("submission_logreg_full_7.csv", index=False)
    print("¡Archivo 'submission_logreg_full_7.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

Entrenando Regresión Logística con el dataset PREPROCESADO...
Generando predicciones sobre el Test...
Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_logreg_full_7.csv' guardado con éxito!
   ID  label
0   0      4
1   1      0
2   2      0
3   3      2
4   4      0


---
## ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST CON GRADIENT CLASSIFIER

---

In [ ]:
# modelos de familia boosting descartado

from sklearn.ensemble import GradientBoostingClassifier

print("Entrenando Gradient Boosting con el dataset PREPROCESADO...")
gb_final = GradientBoostingClassifier(random_state=42)
gb_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = gb_final.predict(X_test_prep)

# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    # Guardamos con un nombre descriptivo
    submission.to_csv("submission_gradient_boosting_full.csv", index=False)
    print("¡Archivo 'submission_gradient_boosting_full.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

Entrenando Gradient Boosting con el dataset PREPROCESADO...
Generando predicciones sobre el Test...
Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_gradient_boosting_full.csv' guardado con éxito!
   ID  label
0   0      4
1   1      0
2   2      0
3   3      1
4   4      0


---
## DOCUMENTACIÓN

---

En esta fase se buscó romper el "techo de cristal" de la iteración anterior mediante una ingeniería de datos avanzada. El resultado fue un éxito rotundo para los modelos sensibles a la escala, validando la hipótesis de que el ruido era el principal limitante.

### 1. 🛠️ Refinamiento del Pipeline (La Clave del Éxito)
Se abandonó la imputación simple y se implementó un pipeline robusto diseñado para limpiar la señal:
* **Limpieza Fisiológica:** Corrección de valores imposibles (ej. ceros en colesterol) antes de imputar.
* **Manejo de Outliers:** Clipping y uso de `RobustScaler` (basado en cuartiles) en lugar de StandardScaler (basado en media/desviación), eliminando la distorsión por valores extremos.
* **Imputación Avanzada (KNN):** Uso de `KNNImputer` (k=15) para reconstruir datos faltantes basándose en la similitud entre pacientes reales.

### 2. 🏆 Resultados en Kaggle (Impacto del Preprocesado)
Los resultados validaron la estrategia de limpieza agresiva:

* **🥇 SVM (0.55978):** **Salto masivo de rendimiento.** Pasó de ser el peor modelo (0.35 en It. 1) a ser el **mejor**. Esto confirma que el algoritmo era correcto, pero los datos "sucios" anteriores le impedían trazar un hiperplano válido.
* **🥈 Regresión Logística (0.54347):** Mantuvo su solidez y mejoró gracias a la imputación KNN, confirmando la fuerte componente lineal del problema.
* **🥉 Gradient Boosting (0.49456):** Se estancó. Al ser un modelo robusto por naturaleza a outliers y nulos, no se benefició tanto de la limpieza avanzada como sus rivales, quedando relegado.

### 3. 📊 Observaciones Críticas
* **El Preprocesado es el Modelo:** La mejora de +0.20 en el SVM demuestra que en este dataset, la ingeniería de características y la limpieza aportan más valor que la complejidad del algoritmo.
* **Validación de Robustez:** `RobustScaler` fue el factor diferencial. Al ignorar los outliers en el escalado, permitió que el SVM y la Regresión Logística vieran la distribución real de los datos.

### 4. 🚀 Próximos Pasos
Con un SVM fuerte (0.56) y un Gradient Boosting estancado (0.49), la estrategia natural es el **Ensamblaje (Stacking)**. Se buscará combinar la precisión geométrica del SVM con la capacidad de los árboles para capturar excepciones no lineales, intentando superar la barrera del 0.57.